In [17]:
import pandas as pd
import numpy as np

val = pd.read_csv("../../../Insight-Wave/StockPrediction/data/Fundamental+Technical Data/STOCK_DATA_WEEKLY/ADVANC.csv")
predict = pd.read_csv("../../../Insight-Wave/StockPrediction/model/ADVANC/Fundamental+LDA News+LDA Twitter+GDELT V1/XGBModel_lags2_lags_past_covariates2_output_chunk_length1.csv")

In [29]:
def calculate_directional_accuracy(actual, forecast):
    return 0

def calculate_mape(actual, forecast):
    """Calculate Mean Absolute Percentage Error, handling zeros in actual values."""
    actual, forecast = np.array(actual), np.array(forecast)
    non_zero_actual = np.where(actual == 0, np.nan, actual)  # Replace zeros with NaN
    mape = np.mean(np.abs((actual - forecast) / non_zero_actual)) * 100
    return np.nanmean(mape)

def calculate_rmse(actual, forecast):
    """Calculate Root Mean Square Error."""
    actual, forecast = np.array(actual), np.array(forecast)
    if len(actual) != len(forecast):
        raise ValueError("The length of actual and forecast arrays must match.")
    mse = np.mean((forecast - actual) ** 2)
    return np.sqrt(mse)

def cal_err_and_acc(predicted, val, condition=True):

    val.reset_index(drop=True, inplace=True)
    predicted.reset_index(drop=True, inplace=True)

    actual = val['Close']
    forecast = predicted['Closing_Price']
    
    mape_error = calculate_mape(actual.iloc[1:], forecast)
    rmse_error = calculate_rmse(actual.iloc[1:], forecast)
    dir_acc = calculate_directional_accuracy(actual, forecast)
    
    if condition:
        return mape_error, rmse_error, dir_acc
    else:
        print(f"MAPE = {mape_error:.2f} %")
        print(f"RMSE = {rmse_error:.2f} %\n")
        print(f"Directional Accuracy = {dir_acc:.2f} %")


In [44]:
val_df = val[['Close']].iloc[-35-35-1:-35]
predict_df = predict[['Closing_Price']]

In [45]:
val_df

,Close
138,216.0
139,228.0
140,220.0
141,223.0
142,223.0
143,237.0
144,233.0
145,232.0
146,232.0
147,231.0


In [46]:
predict_df

,Closing_Price
0,217.92
1,214.12
2,221.95
3,218.08
4,222.10
5,225.09
6,226.79
7,228.41
8,230.12
9,226.50


In [47]:
try:
    avg_mape, avg_rmse, avg_dir = cal_err_and_acc(predict_df, val_df, True)
except Exception as e:
    print("Error when calculate error ", e)

In [48]:
avg_mape

2.808566048579473